# Run the *Heat* model through a SensibleBmi

*Heat* models the diffusion of temperature on a uniform rectangular plate with Dirichlet boundary conditions. View the source code for the [model](https://github.com/csdms/bmi-example-python/blob/master/heat/heat.py) and its [BMI](https://github.com/csdms/bmi-example-python/blob/master/heat/bmi_heat.py) on GitHub.

Start by importing the *Heat* BMI along with `make_sensible`, which we will use to wrap the BMI.

In [ ]:
import matplotlib.pyplot as plt
from heat import BmiHeat

from sensible_bmi.sensible_bmi import make_sensible

Make the `BmiHeat` class more sensible.

In [ ]:
BmiHeat = make_sensible("BmiHeat", BmiHeat)
x = BmiHeat()

Start the *Heat* model through its BMI using a configuration file:

In [ ]:
!cat heat.yaml

In [ ]:
x.initialize("heat.yaml")

In [ ]:
x.name

Check the time information for the model.

In [ ]:
print(x.time)

In [ ]:
print(f"Start time: {x.time.start}")
print(f"End time: {x.time.stop}")
print(f"Current time: {x.time.current}")
print(f"Time step: {x.time.step}")
print(f"Time units: {x.time.units}")

Show the input and output variables for the component (aside on [Standard Names](https://csdms.colorado.edu/wiki/CSDMS_Standard_Names)):

In [ ]:
print(x.input_var_names)
print(x.output_var_names)

Input and output variables are accessed through the `var` attribute.

In [ ]:
print(x.var["plate_surface__temperature"])

print(f"data type: {x.var['plate_surface__temperature'].type}")
print(f"size: {x.var['plate_surface__temperature'].size}")

Next, get the identifier for the grid on which the temperature variable is defined:

In [ ]:
print(x.var["plate_surface__temperature"].grid)

Then get the grid attributes:

In [ ]:
x.grid[0]

In [ ]:
print(f"Grid type: {x.grid[0].type}")
print(f"Grid rank: {x.grid[0].rank}")
print(f"Grid shape: {x.grid[0].shape}")
print(f"Grid spacing: {x.grid[0].spacing}")

Use the variable's `get` method to get an array of its values. Note that the array is flattened.

In [ ]:
temperature = x.var["plate_surface__temperature"].get()

print(f"minimum temperature: {temperature.min()}")
print(f"maximum temperature: {temperature.max()}")
print(f"shape: {temperature.shape}")

In [ ]:
n_rows, n_cols = x.grid[0].shape

plt.imshow(x.var["plate_surface__temperature"].get().reshape((n_rows, n_cols)))

Through the model's BMI, zero out the initial temperature field, except for an impulse near the middle.
Because this variable is both an input and output variable, it has a `set` method. Use this to set the model's values.

In [ ]:
temperature = x.var["plate_surface__temperature"].zeros()

temperature.shape = (n_rows, n_cols)
temperature[n_rows // 2, n_cols // 2] = 100.0

x.var["plate_surface__temperature"].set(temperature)

Check that the temperature field has been updated.

In [ ]:
plt.imshow(x.var["plate_surface__temperature"].get().reshape((n_rows, n_cols)))
print(x.var["plate_surface__temperature"].get().max())

Now advance the model by a single time step:

In [ ]:
x.update()

View the new state of the temperature field:

In [ ]:
print(x.var["plate_surface__temperature"].get().max())

There's diffusion!

Advance the model to some distant time. Notice that the current time is being updated as the model advances in time.

In [ ]:
from tqdm.notebook import tqdm

distant_time = 2000 * x.time.step
with tqdm(total=distant_time) as pbar:
    while x.time.current <= distant_time:
        x.update()

        pbar.update(x.time.current - pbar.n)

View the final state of the temperature field:

In [ ]:
plt.plot(
    x.var["plate_surface__temperature"].get().reshape((n_rows, n_cols))[n_rows // 2]
)

In [ ]:
plt.imshow(x.var["plate_surface__temperature"].get().reshape((n_rows, n_cols)))

Note that temperature isn't conserved on the plate:

End the model:

In [ ]:
x.finalize()